In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv("../data_page1.csv")

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
import matplotlib.pyplot as plt

# Suprimir advertencias de convergencia de ARIMA, etc.
warnings.filterwarnings("ignore")

# Carga de datos
data = pd.read_csv("../data_page1.csv")

row = {}
for i in data.columns:
    row[i] = []

# Diccionario para almacenar los modelos y sus resultados detallados
#model_results = {}

# Definir los órdenes de ARIMA a probar: (p, d, q)
arima_orders = [
    (1, 1, 0), (0, 1, 1), (1, 1, 1),
    (2, 1, 0), (0, 1, 2), (2, 1, 1)
]

for region in data.región.unique():
    # Inicializar el diccionario de resultados para la región actual
    #model_results[region] = {}
    
    for specie in data.specie.unique():
        # Inicializar el diccionario de resultados para la especie actual
        #model_results[region][specie] = {}
        
        # Selección de región y especie
        df = data[(data.región==region) & (data.specie==specie)].copy()
        df.sort_values("year", inplace=True)
        df.reset_index(inplace=True, drop=True)

        if len(df.year.unique()) != df.shape[0] :
            print(f"Hay años repetidos para {region} - {specie}. Saltando.")
            continue

        features_selec = ["carne", "poblacion", "produccion", "rendimiento"]

        # Modelo ARIMA
        for feat in features_selec:
            Y = df[feat]
            
            best_aic = np.inf
            best_order = None
            best_model_fit = None
            
            # --- Bucle para encontrar el mejor orden ARIMA (p, d, q) ---
            for order in arima_orders:
                try:
                    model = ARIMA(Y, order=order)
                    model_fit = model.fit()

                    if model_fit.aic < best_aic:
                        best_aic = model_fit.aic
                        best_order = order
                        best_model_fit = model_fit
                except:
                    continue
            
            # --- Almacenamiento del Modelo y Predicción ---
            prediction = np.nan
            
            if best_model_fit is not None:
                prediction = best_model_fit.forecast(steps=1).iloc[-1]
                
                # 1. Almacenar el modelo ajustado
                '''
                model_results[region][specie][feat] = {
                    "model": best_model_fit,
                    "order": best_order,
                    "aic": best_aic,
                    "data": Y.copy(),
                    "prediction": prediction
                }
                '''
                # 2. Generar el gráfico
                
                # Crear la serie de la predicción para graficar
                pred_index = df["year"].max() + 1
                pred_series = pd.Series([prediction], index=[pred_index])
                
                # Crear el índice de tiempo (años) para los datos históricos
                hist_index = df["year"]
                '''
                plt.figure(figsize=(10, 5))
                plt.plot(hist_index, Y, label='Datos Históricos', marker='o')
                plt.plot(pred_series.index, pred_series.values, 
                         label='Predicción ARIMA', marker='*', markersize=10, linestyle='--')
                
                plt.title(f'Predicción ARIMA ({best_order}) para {feat} en {region} ({specie})')
                plt.xlabel('Año')
                plt.ylabel(feat.capitalize())
                plt.legend()
                plt.grid(True)
                plt.show() # Muestra el gráfico
                '''
                #print(f"Región: {region}, Especie: {specie}, Característica: {feat}, Mejor Orden ARIMA: {best_order}, AIC: {best_aic:.2f}, Predicción: {prediction:.2f}")


            # --- Preparación para el DataFrame final ---
            row[feat].append(prediction)


        row["región"].append(region)
        row["year"].append(df.year.max() + 1)
        row["specie"].append(specie)

# Resultado final
result_df = pd.DataFrame(row)


In [3]:
result_df.to_csv("prediction_2025.csv")